In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.1,
                 model_name='gpt-4o',
                 )


In [3]:
# 이미지 분류
from openai import OpenAI

Image_classfy = OpenAI()
response = Image_classfy.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": '''
         너는 사진을 보면 그 물체가 어떤 재활용품인지 알 수 있는 대한민국의 분리배출 전문가야. 나는 물체가 어떤 재활용품인지 잘 몰라서 분리배출을 잘 못하는 일반인이야. 너의 답변을 통해 일반인이 올바른 분리배출을 할 수 있도록 정확한 답변을 해줘.
소개나 인사말 등 답변과 관계 없는 말은 하지 마.
이 사진속 물체가 무엇인지, 상태는 어떤지 확인해서 반드시 목록 중 하나로 선택해서 대답해줘.
###
물체 목록 : 더러운 플라스틱, 깨끗한 플라스틱, 더러운 스티로폼, 깨끗한 스티로폼, 더러운 유색 페트병, 깨끗한 유색 페트병, 더러운 유리병, 깨진 유리병, 종이, 깨끗한 비닐, 더러운 비닐, 금속캔, 깨끗한 투명 페트병, 더러운 투명 페트병
###

인식한 물체와 비슷한 항목이 물체 목록에 없는 경우에는 반드시 예외 답변 예시에 맞게 답변해줘.
### 
예외 답변 양식: '인식한 물체명' 재활용불가
###
         
         '''},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://i.imgur.com/pokscbm.png",
          },
        },
      ],
    }
  ],
  max_tokens=1000,
)
print(response.choices[0].message.content)

인식결과 = response.choices[0].message.content

깨진 유리병


In [4]:
# 지역별 분리배출 안내페이지 (서울시 강남구 등은 현재 pdf 읽어오기가 안되어 보류)
loc_doc = {'서울시 강동구': ('https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_003_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_002', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_003'), '서울시 송파구': ('https://www.songpa.go.kr/www/contents.do?key=3153', 'https://www.songpa.go.kr/www/contents.do?key=3157&', 'https://www.songpa.go.kr/www/contents.do?key=3161&', 'https://www.songpa.go.kr/www/contents.do?key=2117&', 'https://www.songpa.go.kr/www/contents.do?key=3164&', 'https://www.songpa.go.kr/www/contents.do?key=3171&'), '서울시 강서구': ('https://www.gangseo.seoul.kr/env/env010101', 'https://www.gangseo.seoul.kr/env/env010202', 'https://www.gangseo.seoul.kr/env/env010203', 'https://www.gangseo.seoul.kr/env/env010401')}

In [16]:
# RAG
# 지역별 분리배출 안내 페이지 중 사용자가 선택한 지역의 페이지를 로드

from langchain.document_loaders import WebBaseLoader

loc = '서울시 강동구'
data = ''

if loc in loc_doc.keys():
    print(loc_doc[loc])
    loader = WebBaseLoader(loc_doc[loc])
    data = loader.load()
    data
else:
    pass

('https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_003_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_001', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_002', 'https://www.gangdong.go.kr/web/newportal/contents/gdp_005_004_010_004_003')


In [17]:
# 페이지 내부 내용(Text)를 사이즈에 맞게 스플릿 후 저장

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
texts = text_splitter.split_documents(data)


In [18]:
# 페이지 내부 내용을 벡터스토어에 저장


import tiktoken
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

persist_directory = f"./storage/{loc}"
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()


In [19]:
# 벡터스토어 내용을 검색에 활용

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever()
llm = ChatOpenAI(temperature=0.1,
                 max_tokens=2048,
                 model_name='gpt-4o',
                 )
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [20]:
if "재활용불가" in 인식결과:
    print(f"해당 제품은 {인식결과[:인식결과.find(" ")]}이네요! 재활용이 불가합니다. 일반쓰레기로 불출해주세요.")  # 이미지 분류에서 재활용 불가로 나오는 경우 처리
else:
    query = f'''
    ###Prompt 너는 대한민국의 분리배출 전문가야. 나는 분리배출 방법을 잘 모르는 일반인이야. 일반인이 분리배출을 올바르게 할 수 있도록 답변을 작성해줘. 소개나 인사말, 끝인사 등 답변과 관계 없는 말은 절대로 하지 마.
    # {loc}에서 {인식결과} 분리배출 방법을 설명해줘. 반드시 답변 양식에 맞게 답해주고, 내용은 물품명과 분리배출 방법에 따라 수정해서 작성해줘. 

    ###
    답변 양식
    안녕하세요.
    작은 실천을 통해 환경을 지키는 WasteWise입니다.
    
    오늘의 분리배출 지역은 {loc}이시군요.

    해당 제품은 {인식결과}이네요! + 배출 방법을 안내해드릴게요.

    {인식결과.split()[-1]}은
    - 스티로폼에 붙어있는 음식물 찌꺼기나 이물질을 깨끗이 제거해주세요.
    - 스티로폼을 다른 재활용품과 섞이지 않도록 분리하여 배출해주세요.
    - 스티로폼을 가능한 한 작게 부수어 부피를 줄여주세요.
    ###
    '''
    try:
        llm_response = qa(query)
        print(llm_response["result"])
        
        
    except Exception as err:
        print('Exception occurred. Please try again', str(err))


안녕하세요.
작은 실천을 통해 환경을 지키는 WasteWise입니다.

오늘의 분리배출 지역은 서울시 강동구이시군요.

해당 제품은 깨진 유리병이네요! 배출 방법을 안내해드릴게요.

깨진 유리병은
- 깨진 유리병은 재활용이 불가능하므로 일반종량제봉투에 담아 배출해주세요.
- 유리병 조각이 날카로워 다칠 수 있으니 두꺼운 종이나 신문지로 감싸서 안전하게 처리해주세요.
- 유독성 물질이나 불에 타지 않는 재질이므로 특수종량제봉투(pp마대)를 사용해도 됩니다.
